In [ ]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [ ]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    if erp_path == 'frn':
        epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))
    elif erp_path == 'lrp':
        epochs = epochs.apply_baseline(baseline = (-1.3, -1.0))
    elif erp_path == 'ern':
        epochs = epochs.apply_baseline(baseline = (-1.35, -1.05))

    return epochs, pp_directory

In [ ]:
#get index of trials from behavioral data
#rdm before rot and rdm before mir are read in as two different behavioral data
def epoch_index(task, listval, pp, target = np.nan):
    if task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ROT_learningcurve_degrees.csv')
    elif task == 'rdmrot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\RDMROT_learningcurve_degrees_%02d.csv' % target)
    elif task == 'rdmmir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\RDMMIR_learningcurve_degrees_%02d.csv' % target)
    elif task == 'aln':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ALIGNED_learningcurve_degrees.csv')
    elif task == 'mir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\MIR_learningcurve_degrees_%02d.csv' % target)
   
    ppdata = list(df.iloc[:,pp+1])
    
    index = [i for i,v in enumerate(ppdata) if listval[0] < v < listval[1] or listval[2] < v < listval[3]]
    
    #epochs_out = epochs[index]
    numtrials = len(index)
    
    #return epochs_out, numtrials
    return index, numtrials

In [ ]:
#separate function to get indexed epochs for random, since we have them before rotation or mirror
def trial_list_rdm(task, pp, index):
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rdmrot':
            #get trial after first task (trial 48 to 95, 234 to 281)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
                    
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
    elif pp % 2 == 1:   
        if task == 'rdmrot':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

    
    return(triallist)
    

In [ ]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first task (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first five tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [ ]:
#save as evoked objects - runs for any erp type
#each perturbation condition can have separate loops

#cutoffs for small and large errors in rot
conditions = [[15, 27, 33, 45], [-15, 15, 45, 75]]
#separate cutoffs for rdm
conditions_rdm_sml = [[0, 12, 18, 30], [-30, 0, 30, 60]]
conditions_rdm_mid = [[10, 22, 28, 40], [-20, 10, 40, 70]]
conditions_rdm_lrg = [[20, 32, 38, 50], [-10, 20, 50, 80]]
#separate cutoffs per target of mirror reversed trials
conditions_mir_sml = [[0, 12, 18, 30], [-30, 0, 30, 60]]
conditions_mir_mid = [[15, 27, 33, 45], [-15, 15, 45, 75]]
conditions_mir_lrg = [[30, 42, 48, 60], [0, 30, 60, 90]]

conditionnames = ['small', 'lrg']
#within X degrees is considered a "hit"
err_cutoff = [-3, 0, 0, 3]

aln_ntrials = []
rot_sml_ntrials = []
rot_lrg_ntrials = []
rdmrot_ntr_sml = []
rdmrot_ntr_lrg = []
rdmmir_ntr_sml = []
rdmmir_ntr_lrg = []
mir_ntr_sml = []
mir_ntr_lrg = []

# ALIGNED
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    idx, numtrials = epoch_index('aln', err_cutoff, pp)
    aln_ntrials.append(numtrials)

# ROTATION
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        idx, numtrials = epoch_index('rot', conditions[cond], pp)
        if cond == 0:
            rot_sml_ntrials.append(numtrials)
        elif cond == 1:
            rot_lrg_ntrials.append(numtrials)


# RANDOM (could be optimized, but in case we want to separate rdm before rot or mir, we have it as below)
for pp in participants:
    #rdmrot
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    idx_sml = []
    ntr_sml = []
    idx_lrg = []
    ntr_lrg = []
    for cond in range(0, len(conditions_rdm_sml)):
        idx, numtrials = epoch_index('rdmrot', conditions_rdm_sml[cond], pp, target = 15)
        if cond == 0:
            idx_sml.append(idx)
            ntr_sml.append(numtrials)
        elif cond == 1:
            idx_lrg.append(idx)
            ntr_lrg.append(numtrials)
    for cond in range(0, len(conditions_rdm_mid)):
        idx, numtrials = epoch_index('rdmrot', conditions_rdm_mid[cond], pp, target = 25)
        if cond == 0:
            idx_sml.append(idx)
            ntr_sml.append(numtrials)
        elif cond == 1:
            idx_lrg.append(idx)
            ntr_lrg.append(numtrials)
    for cond in range(0, len(conditions_rdm_lrg)):
        idx, numtrials = epoch_index('rdmrot', conditions_rdm_mid[cond], pp, target = 35)
        if cond == 0:
            idx_sml.append(idx)
            ntr_sml.append(numtrials)
        elif cond == 1:
            idx_lrg.append(idx)
            ntr_lrg.append(numtrials)
            
    all_idx_sml = idx_sml[0]+idx_sml[1]+idx_sml[2]
    all_ntr_sml = ntr_sml[0]+ntr_sml[1]+ntr_sml[2]
    all_idx_lrg = idx_lrg[0]+idx_lrg[1]+idx_lrg[2]
    all_ntr_lrg = ntr_lrg[0]+ntr_lrg[1]+ntr_lrg[2]
    
    #sort index
    all_idx_sml.sort()
    all_idx_lrg.sort()
    
    rdmrot_triallist_sml = trial_list_rdm('rdmrot', pp, all_idx_sml)
    rdmrot_triallist_lrg = trial_list_rdm('rdmrot', pp, all_idx_lrg)
    
    rdmrot_ntr_sml.append(all_ntr_sml)
    rdmrot_ntr_lrg.append(all_ntr_lrg)
    
    #rdmmir
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    idx_sml = []
    ntr_sml = []
    idx_lrg = []
    ntr_lrg = []
    for cond in range(0, len(conditions_rdm_sml)):
        idx, numtrials = epoch_index('rdmmir', conditions_rdm_sml[cond], pp, target = 15)
        if cond == 0:
            idx_sml.append(idx)
            ntr_sml.append(numtrials)
        elif cond == 1:
            idx_lrg.append(idx)
            ntr_lrg.append(numtrials)
    for cond in range(0, len(conditions_rdm_mid)):
        idx, numtrials = epoch_index('rdmmir', conditions_rdm_mid[cond], pp, target = 25)
        if cond == 0:
            idx_sml.append(idx)
            ntr_sml.append(numtrials)
        elif cond == 1:
            idx_lrg.append(idx)
            ntr_lrg.append(numtrials)
    for cond in range(0, len(conditions_rdm_lrg)):
        idx, numtrials = epoch_index('rdmmir', conditions_rdm_mid[cond], pp, target = 35)
        if cond == 0:
            idx_sml.append(idx)
            ntr_sml.append(numtrials)
        elif cond == 1:
            idx_lrg.append(idx)
            ntr_lrg.append(numtrials)
            
    all_idx_sml = idx_sml[0]+idx_sml[1]+idx_sml[2]
    all_ntr_sml = ntr_sml[0]+ntr_sml[1]+ntr_sml[2]
    all_idx_lrg = idx_lrg[0]+idx_lrg[1]+idx_lrg[2]
    all_ntr_lrg = ntr_lrg[0]+ntr_lrg[1]+ntr_lrg[2]
    
    #sort index
    all_idx_sml.sort()
    all_idx_lrg.sort()
    
    rdmmir_triallist_sml = trial_list_rdm('rdmmir', pp, all_idx_sml)
    rdmmir_triallist_lrg = trial_list_rdm('rdmmir', pp, all_idx_lrg)
    
    #combine both rdmrot and rdmmir accdg to conditions
    triallist_sml = rdmrot_triallist_sml + rdmmir_triallist_sml
    triallist_lrg = rdmrot_triallist_lrg + rdmmir_triallist_lrg
    triallist_sml.sort()
    triallist_lrg.sort()

    rdmmir_ntr_sml.append(all_ntr_sml)
    rdmmir_ntr_lrg.append(all_ntr_lrg)

# MIRROR
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    idx_sml = []
    ntr_sml = []
    idx_lrg = []
    ntr_lrg = []
    for cond in range(0, len(conditions_mir_sml)):
        idx, numtrials = epoch_index('mir', conditions_mir_sml[cond], pp, target = 15)
        if cond == 0:
            idx_sml.append(idx)
            ntr_sml.append(numtrials)
        elif cond == 1:
            idx_lrg.append(idx)
            ntr_lrg.append(numtrials)
    for cond in range(0, len(conditions_mir_mid)):
        idx, numtrials = epoch_index('mir', conditions_mir_mid[cond], pp, target = 30)
        if cond == 0:
            idx_sml.append(idx)
            ntr_sml.append(numtrials)
        elif cond == 1:
            idx_lrg.append(idx)
            ntr_lrg.append(numtrials)
    for cond in range(0, len(conditions_mir_lrg)):
        idx, numtrials = epoch_index('mir', conditions_mir_mid[cond], pp, target = 45)
        if cond == 0:
            idx_sml.append(idx)
            ntr_sml.append(numtrials)
        elif cond == 1:
            idx_lrg.append(idx)
            ntr_lrg.append(numtrials)
            
    all_idx_sml = idx_sml[0]+idx_sml[1]+idx_sml[2]
    all_ntr_sml = ntr_sml[0]+ntr_sml[1]+ntr_sml[2]
    all_idx_lrg = idx_lrg[0]+idx_lrg[1]+idx_lrg[2]
    all_ntr_lrg = ntr_lrg[0]+ntr_lrg[1]+ntr_lrg[2]
    
    #sort index
    all_idx_sml.sort()
    all_idx_lrg.sort()
    
    mir_ntr_sml.append(all_ntr_sml)
    mir_ntr_lrg.append(all_ntr_lrg)

In [ ]:
#save number of trials per error condition into data frame
err_cond_ntrials = pd.DataFrame(
    {'aln_hits': aln_ntrials,
     'rot_sml': rot_sml_ntrials,
     'rot_lrg': rot_lrg_ntrials,
     'rdmrot_sml': rdmrot_ntr_sml,
     'rdmrot_lrg': rdmrot_ntr_lrg,
     'rdmmir_sml': rdmmir_ntr_sml,
     'rdmmir_lrg': rdmmir_ntr_lrg,
     'mir_sml': mir_ntr_sml,
     'mir_lrg': mir_ntr_lrg
    })

newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'NTrials_Conditions.csv')
err_cond_ntrials.to_csv(newdf_filename)